## Exercise 10

Date: 11/16/2021 <br>
Group Name: Broken Toes <br>
Group Members: Emma Bright, Mike Santoro, Duncan Ferguson


***

In [ ]:
import numpy as np
import math
import pandas as pd
from sklearn import tree
from sklearn.naive_bayes import *
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.inspection import permutation_importance
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

### Part 1:  PCA

Principle Component Analysis (PCA) can be used to reduce the time needed to build machine learning models.  You are to apply PCA the weather prediction data set using GausianNP.
First create a table (or chart) of the explained_variance_ratio_ for the data set.  Then run your models for 1, 2, 3, 4, 5, 6, 7, 8, and all components.  Create a table (or chart) showing the accuracy as a function of the number of  components.

Create a function to perform the Gaussian NB machine learning algorithm.

In [ ]:
def runGaussianNB(x_train, x_test, y_train, y_test):
	global gausianNB_predicted
	model = GaussianNB()
	model.fit(x_train,y_train)
	gausianNB_predicted = model.predict(x_test)
	accuracy = accuracy_score(y_test, gausianNB_predicted)
	imps = permutation_importance(model, x_test, y_test)

Read in the weather data.

In [ ]:
df = pd.read_csv('cleanInfile.csv')
df.head()

Create a table of the Explained Variance Ratio for the dataset.

In [ ]:

Y = df['RainTomorrow']
df2 = df.drop('RainTomorrow',axis=1)

x_train, x_test, y_train, y_test  = train_test_split( df2, Y,  test_size=0.20, random_state=1)
save_xtrain = x_train
save_xtest = x_test
save_ytrain = y_train
save_ytest = y_test
runGaussianNB(x_train, x_test, y_train, y_test)

pca = PCA()
x_train = pca.fit_transform(save_xtrain)
x_test = pca.fit_transform(save_xtest)
explained_variance = pca.explained_variance_ratio_
print("explained_variance = ")
print(explained_variance)

Run the Gaussian NB model for 1, 2, 3, 4, 5, 6, 7, 8, and all components.

In [ ]:
accuracy_list= []
x_train = save_xtrain
x_test = save_xtest
# Run with all components
runGaussianNB(x_train, x_test, y_train, y_test)
accuracy_list.append(['all', accuracy_score(y_test, gausianNB_predicted)])


# Run with 1, 2, 3....., 8 components
for i in range (1, 9):
    pca = PCA(n_components=i)
    x_train = pca.fit_transform(save_xtrain)
    x_test = pca.fit_transform(save_xtest)
    runGaussianNB(x_train, x_test, y_train, y_test)
    accuracy_list.append( [str(i), accuracy_score(y_test, gausianNB_predicted)])

result = pd.DataFrame(accuracy_list, columns=["Number of Components", "Accuracy"])
result


#### Based on this, how many components do you recommend using for constructing the GausianNB model for this data set?

Based on this output I would recommend using 4 components for the Gaussian Naive Bayes model as this has the highest level of accuracy.

***

### Part 2: Clustering Quality

Apply kmeans and dbmeans clustering to these labeled data sets - outfile1, outfile2, and outfile3.
Calculate the following three metrics on the clustering for kmeans and dbscan:  homogeneity, completeness, and adjusted_mutual_info_score.

In [ ]:
numClusters = 3

outfile = pd.read_csv('outfile.csv')
labels1 = pd.read_csv('outfile1.csv')
labels2 = pd.read_csv('outfile2.csv')
labels3 = pd.read_csv('outfile3.csv')

kmeans = KMeans(n_clusters=numClusters, random_state=0).fit(outfile)
db = DBSCAN(eps=1.5, min_samples=4).fit(outfile)

In [ ]:
outfile['predKM'] = kmeans.labels_
outfile['predDB'] = db.labels_

In [ ]:
outfile.head()

In [ ]:
labels1['actual'] = 0
labels2['actual'] = 1
labels3['actual'] = 2
labels = labels1.append(labels2, ignore_index = True)
labels = labels.append(labels3, ignore_index = True)
labels.head()

In [ ]:
df = pd.merge(labels, outfile,  how='left', left_on=['a1','a2'], right_on = ['a1','a2'])
df


In [ ]:
print('K-Means')
print(f"Homogenity Score: {homogeneity_score(df.actual.to_list(),df.predKM.to_list())}")
print(f'Completeness Score: {completeness_score(df.actual.to_list(), df.predKM.to_list())}')
print(f'Adjusted Mutual Info Score: {adjusted_mutual_info_score(df.actual.to_list(), df.predKM.to_list())}')

In [ ]:
print('DB Scan')
print(f"Homogenity Score: {homogeneity_score(df.actual.to_list(),df.predDB.to_list())}")
print(f'Completeness Score: {completeness_score(df.actual.to_list(), df.predDB.to_list())}')
print(f'Adjusted Mutual Info Score: {adjusted_mutual_info_score(df.actual.to_list(), df.predDB.to_list())}')

#### A description of what these metrics tell you about the clustering?

Homogeneity Score is the measure that each cluster contains only members of a single class. <br>
The Completeness Score is a measure that all members of a given class are assigned to the same cluster. The two previous metrics can be combined to obtain the Adjusted Mutual Info Score since both are bounded below by 0.0 and above by 1.0 (higher is better).<br>
We can see from that excercise that DB-scan was much more effective at clustering this data-set as all of the scores for the metrics are much closer to 1 than those of K-Means.